Algrotimo Genetico enfocado a el juego de un laberinto, utilizando algoritmos geneticos se tratara de buscar un camino optimo en un laberitnto. EL suiguiente codigo se hizo con ayuda de chat (gpt)

Nombres:José Fernando
Apellidos:Alfaro Ayzama

In [ ]:
import numpy as np
import random

# Definición del laberinto (0 representa espacio libre y 1 representa pared)
laberinto = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0]
])

# Punto inicial y final del laberinto (fila, columna)
inicio = (0, 0)
final = (6, 5)

# Parámetros del algoritmo genético
num_generaciones = 100
tam_poblacion = 50
prob_mutacion = 0.2
#creacion de individuo
def generar_camino_aleatorio():
    camino = [] #se almacena el caminno
    pos_actual = inicio #se controla la poscicion actual
    visitados = set() #mantiene el registro de las posiciones visitadas

    while pos_actual != final:#mientras la pos actual se distinta de la final seguira iterando
        movimientos_posibles = obtener_movimientos_posibles(pos_actual, visitados)#se controla los movimientos que puede realizar
        if not movimientos_posibles:#si ya no hay mas movimientos posibles no retorna nada
            return None
        siguiente_movimiento = random.choice(movimientos_posibles)#seleccciona un moviento aleatorio de los movimientos posibles
        camino.append(siguiente_movimiento)#añade el moviento al camino
        pos_actual = siguiente_movimiento #actualiza la pos actual
        visitados.add(pos_actual) #la posicion actual se marca como visitada
    return camino #retorna el camino completo

def obtener_movimientos_posibles(posicion, visitados): #se recive una posicion y la lista de visitados
    movimientos = [] #lisata donde se almacenan los movimientos posibles
    filas, columnas = laberinto.shape #optenemos el numero de filas y columnas del laberinto
    fila, columna = posicion #la posicion en filas y columnas
    #todas verifican si hay pared o si ya esta visitado.
    #verifica si es posile moverse hacia arriba
    if fila > 0 and laberinto[fila - 1, columna] == 0 and (fila - 1, columna) not in visitados:
        movimientos.append((fila - 1, columna))
    #verifica si puede moverse hacia abajo
    if fila < filas - 1 and laberinto[fila + 1, columna] == 0 and (fila + 1, columna) not in visitados:
        movimientos.append((fila + 1, columna))
    #Verifica si se puede mover hacia la izquierda
    if columna > 0 and laberinto[fila, columna - 1] == 0 and (fila, columna - 1) not in visitados:
        movimientos.append((fila, columna - 1))
    #verifica si se puede mover a la derecha
    if columna < columnas - 1 and laberinto[fila, columna + 1] == 0 and (fila, columna + 1) not in visitados:
        movimientos.append((fila, columna + 1))

    return movimientos #retorna la lista de posibles movimientos que se puede realizar

def calcular_fitness(camino): #el fitnes se calcula contando las posiones que tine el camino mientras menos posiciones mejor fitnes
    return 1 / (len(camino) + 1)

def cruzar_padres(padre1, padre2): #se le pasa 2 individuos
    punto_cruce = random.randint(1, len(padre1) - 1)#se selecciona un punto aleatorio para el cruce, a lo largo del padre 1
    hijo = padre1[:punto_cruce] + padre2[punto_cruce:]#desde el pprincipio hasta el punto padre1 y desde el punto al final padre2
    return hijo #se retorna reslutado del nuevo hijo

def mutar_camino(camino, prob_mutacion): #recive el camino y prob de mutacion
    for i in range(len(camino)): #se itera a lo largo del camino en este caso individuo
        if random.random() < prob_mutacion: #se saca un random entre 0 y 1 y si es menor entra
            camino[i] = (random.randint(0, laberinto.shape[0] - 1), random.randint(0, laberinto.shape[1] - 1)) #en un punto del camino se le asigna aleatoriamente dos coordenadas
    return camino #camino mutado                                                                               #controladas con el num de col y fil del laberinto

def generar_poblacion_inicial(tam_poblacion): #recive el tamaño de poblacion que tendra
    poblacion = [] #se almacena la poblacion
    while len(poblacion) < tam_poblacion: #mientras la poblacion sea menor a el tam de poblacion
        individuo = generar_camino_aleatorio() #un individuo es un camino generado anteriormente
        if individuo is not None:  #verifica que el individuo no sea NONE
         poblacion.append(individuo) #agrega el individuo a la poblacion
    return poblacion #retorna la poblacion con los individuos
def seleccion_por_torneo(poblacion, tam_torneo): #recibe la poblacion y el numero de individuos que entraran en el torneo
    torneo = random.sample(poblacion, tam_torneo)#se escoge aletoriamente (n=tamtorneo) participantes(individuos) de la poblacion
    return max(torneo, key=calcular_fitness) #se escoge al que tengo mayor fitnes de todos los individuos seleccionados


def algoritmo_genetico(num_generaciones, tam_poblacion, tam_torneo, prob_mutacion):
    poblacion = generar_poblacion_inicial(tam_poblacion) #se crea la poblacion

    for generacion in range(num_generaciones): #se iteria para crear las generaciones
        poblacion = sorted(poblacion, key=lambda x: calcular_fitness(x), reverse=True)#la poblacion se ordena de forma descendente por su fitnest
        if calcular_fitness(poblacion[0]) == 1: #se verifica si el mejor individuo de esa poblacion es igual a 1
            break #si es asi ya se encontro la mejor ruta y sale del bucle
        nueva_generacion = poblacion[:2] #guarda los dos mejores individuos de la poblacion, porq serian lo mejor de la poblacion y se gurdan para la suiguiente generacion

        for i in range(tam_poblacion - 2): #el bucle se repite para llenar la pobalcion con individuos generados por selccion y cruzamiento
            padre1 = seleccion_por_torneo(poblacion, tam_torneo) #seleccionando  padres mediante la selaccion por torneo
            padre2 = seleccion_por_torneo(poblacion, tam_torneo) #para obtner mejores los 2 mejores individuos
            nuevo_individuo = cruzar_padres(padre1, padre2) #para realizar la cruza de estos
            nueva_generacion.append(nuevo_individuo) #se agrega el nuevo indivuo resultado de la cruza para mejorar la exploracion de resultados

        for i in range(2, len(nueva_generacion)): #itera sobre los nuevos individuos de la nueva poblacion excluyendo los dos primeros
            nueva_generacion[i] = mutar_camino(nueva_generacion[i], prob_mutacion) #aplica mutaciones a los indivuos restantes

        poblacion = nueva_generacion #se actuliza la poblacion con la nueva generacion creada.
        #el bucle continua hasta que se generen todas las generaciones si no se encuentra el mejor fitnest antes
    return poblacion[0]# se retorna el primer individuo de la ultima generacion que deberia ser el que tenga mejor fitnes

# Ejecución del algoritmo genético
solucion = algoritmo_genetico(num_generaciones, tam_poblacion, tam_torneo=10, prob_mutacion=0.2)
print("Mejor camino:")
print(solucion)

Mejor camino:
[(1, 0), (2, 0), (3, 0), (3, 1), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 5), (6, 5)]


In [ ]:
laberinto = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0]
])